*Technical University of Munich<br>
Professorship of Environmental Sensing and Modeling<br><br>*
**Author:**  Daniel Kühbacher<br>
**Date:**  07.05.2024

--- 

# Detector Emission Calculation

<!--Notebook description and usage information-->
HBEFA provides different emission factors for different traffic situations e.g. Freeflow, Saturated or Stop&Go. These traffic situations are characterized by different average speeds of the vehicles, which is also provided by the traffic detectors.


In [402]:
import sys

import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import time

sys.path.append('../utils')
import data_paths
from hbefa_hot_emissions import HbefaHotEmissions

# Import Data

In [403]:
# import speed and volume data from traffic counting stations
visum_links = gpd.read_file(data_paths.VISUM_FOLDER_PATH + 'visum_links.gpkg')

# raw speed and counting data from LHM counting stations
cnt_lhm = pd.read_parquet(data_paths.MST_COUNTING_PATH + 'preprocessed_lhm_counting_data.parquet')
# only detectors that are assigned to a road link in the visum model
cnt_lhm = cnt_lhm[cnt_lhm['road_link_id'].isin(visum_links['road_link_id'].unique())] 
cnt_lhm_2019 = cnt_lhm[cnt_lhm['date'].between('2019-01-01','2019-12-31')] # reduce to 2019 data

# import hbefa emission factors
hbefa = HbefaHotEmissions()

Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_PC.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_LCV.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_HGV.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_Coach.XLS
Loaded emission factors from /Users/daniel_tum/Documents/projects/traffic inventory v2/traffic-emission-inventory/data/restricted_input/hbefa/EFA_HOT_Vehcat_MOT.XLS


# Prepare dataframe with detector information

In [404]:
columns_to_merge = ['road_type', 'hbefa_gradient',
                    'hbefa_speed', 'speed', 'road_link_id']

_det = cnt_lhm_2019.set_index('detector_id')['road_link_id']\
    .reset_index()\
    .drop_duplicates()

det_info = pd.merge(_det, visum_links[columns_to_merge], 
                    left_on = 'road_link_id', 
                    right_on = 'road_link_id', 
                    how = 'inner').drop_duplicates()

# set duplicates in road gradient to 0%
det_info = det_info.groupby('detector_id').agg({'road_link_id': 'first', 
                                                'road_type': 'first', 
                                                'speed': 'first', 
                                                'hbefa_speed': 'first', 
                                                'hbefa_gradient': lambda x: x.iloc[0] if len(x)==1 else '0%'})
det_info.head()

,road_link_id,road_type,speed,hbefa_speed,hbefa_gradient
detector_id,,,,,
4010011,80645.0,TrunkRoad/Primary-City,50,50,0%
4010012,80645.0,TrunkRoad/Primary-City,50,50,0%
4010013,80645.0,TrunkRoad/Primary-City,50,50,0%
4010014,80645.0,TrunkRoad/Primary-City,50,50,0%
4010021,80645.0,TrunkRoad/Primary-City,50,50,0%


# Prepare speed and volume dataframe

In [405]:
hour_value_columns = [str(x) for x in range(0,24)]

_cnt = cnt_lhm_2019.melt(id_vars = ['date', 'detector_id', 'vehicle_class', 'metric'],
                         value_vars =hour_value_columns)

_cnt['timestamp'] = _cnt.apply(lambda row: pd.Timestamp.combine(row['date'],
                                                                time(int(row['variable']))), axis =1)
_cnt = _cnt[(_cnt['vehicle_class']!='SUM')]

In [406]:
# prepare speed dataset
_cnt_speed = _cnt[(_cnt['metric'] == 'speed') &
                  (_cnt['value'] != 0)][['detector_id',
                                         'timestamp',
                                         'value']]
_cnt_speed = _cnt_speed.rename(columns = {'value': 'measured_speed'})

# prepare volume dataset
_cnt_volume = _cnt[_cnt['metric'] == 'volume']
_cnt_volume = _cnt_volume.pivot(index = ['detector_id', 'timestamp'],
                                columns = 'vehicle_class',
                                values = 'value')
_cnt_volume['SUM_PCU'] = _cnt_volume.mul(pd.Series(hbefa.car_unit_factors)).sum(axis = 1)

# combine speed and volume dataset
cnt_base = pd.merge(_cnt_speed, _cnt_volume,
                    left_on = ['detector_id', 'timestamp'], 
                    right_index = True, 
                    how = 'inner')

# add detector information
detector_dat = pd.merge(cnt_base, det_info,
                        left_on='detector_id',
                        right_on='detector_id')
detector_dat.head()

,detector_id,timestamp,measured_speed,BUS,HGV,LCV,MOT,PC,SUM_PCU,road_link_id,road_type,speed,hbefa_speed,hbefa_gradient
0,4010011,2019-01-03,48.0,0.0,2.0,1.0,0.0,37.0,43.0,80645.0,TrunkRoad/Primary-City,50,50,0%
1,4010011,2019-01-04,50.0,0.0,0.0,2.0,0.0,58.0,60.0,80645.0,TrunkRoad/Primary-City,50,50,0%
2,4010011,2019-01-05,48.0,0.0,4.0,3.0,0.0,84.0,97.0,80645.0,TrunkRoad/Primary-City,50,50,0%
3,4010011,2019-01-06,44.0,0.0,0.0,7.0,0.0,69.0,76.0,80645.0,TrunkRoad/Primary-City,50,50,0%
4,4010011,2019-01-07,49.0,0.0,2.0,2.0,0.0,33.0,40.0,80645.0,TrunkRoad/Primary-City,50,50,0%


# Calculate LOS by speed distribution

In [407]:
# import LOS speed allocation
path = data_paths.EF_path + 'ef_eval/v_per_TraSit.XLS'
ef_speed = pd.read_excel(path)
ef_speed['area_type'] = ef_speed.apply(lambda row: row['TS'].split('/')[0], axis = 1)
ef_speed['road_type'] = ef_speed.apply(lambda row: row['TS'].split('/')[1], axis = 1)
ef_speed['speed'] = ef_speed.apply(lambda row: row['TS'].split('/')[2], axis = 1)
ef_speed['TraSit'] = ef_speed.apply(lambda row: row['TS'].split('/')[3], axis = 1)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [409]:
def get_true_los(ef_speed_df:pd.DataFrame,
                 allowed_speed:float,
                 measured_speed:float,
                 road_type:str,
                 area_type:str = 'URB')-> str:

    df = ef_speed_df
    df_sub = df[(df['RoadCat'] == 'Urban') &
         (df['road_type'] == road_type) &
         (df['speed'] == str(allowed_speed)) & 
         (df['area_type'] == area_type)]

    absolute_diff = (df_sub['pass_ car'] - measured_speed).abs()
    return df.loc[absolute_diff.idxmin(), 'TS']


In [410]:
# apply function
detector_dat['true_ts'] = detector_dat.apply(lambda row: get_true_los(ef_speed_df=ef_speed,
                                                                      allowed_speed=row['speed'],
                                                                      measured_speed=row['measured_speed'],
                                                                      road_type=hbefa.hbefa_road_abbreviations[row['road_type']]
                                                                      ), axis = 1)
detector_dat.head()

,detector_id,timestamp,measured_speed,BUS,HGV,LCV,MOT,PC,SUM_PCU,road_link_id,road_type,speed,hbefa_speed,hbefa_gradient,true_ts
0,4010011,2019-01-03,48.0,0.0,2.0,1.0,0.0,37.0,43.0,80645.0,TrunkRoad/Primary-City,50,50,0%,URB/Trunk-City/50/Freeflow
1,4010011,2019-01-04,50.0,0.0,0.0,2.0,0.0,58.0,60.0,80645.0,TrunkRoad/Primary-City,50,50,0%,URB/Trunk-City/50/Freeflow
2,4010011,2019-01-05,48.0,0.0,4.0,3.0,0.0,84.0,97.0,80645.0,TrunkRoad/Primary-City,50,50,0%,URB/Trunk-City/50/Freeflow
3,4010011,2019-01-06,44.0,0.0,0.0,7.0,0.0,69.0,76.0,80645.0,TrunkRoad/Primary-City,50,50,0%,URB/Trunk-City/50/Freeflow
4,4010011,2019-01-07,49.0,0.0,2.0,2.0,0.0,33.0,40.0,80645.0,TrunkRoad/Primary-City,50,50,0%,URB/Trunk-City/50/Freeflow


# Calculate emission for each detector loop

This can be peformed based on the traffic counting data driven estimation of the true traffic situation.

In [411]:
def calculate_emissions(vehicle_class,
                        vehicle_volume,
                        los_class,
                        hbefa_gradient, 
                        component):
    try:
        ef = hbefa.ef_dict[vehicle_class]['EFA_weighted']\
            [2019, los_class, hbefa_gradient, component]
        return vehicle_volume * ef
    except:
        return np.nan
                                                                                       
for c in hbefa.components: 
    for v in hbefa.vehicle_classes:
        detector_dat[f'{c}_{v}'] = detector_dat.apply(lambda row: calculate_emissions(vehicle_class = v,
                                                                                      vehicle_volume = row[v],
                                                                                      los_class = row['true_ts'],
                                                                                      hbefa_gradient= row['hbefa_gradient'],
                                                                                      component= c), axis = 1)

# Save detector emission calculation to file

In [412]:
filename = data_paths.COUNTING_PATH + 'detector_emissions_v_ts.feather'
detector_dat.to_feather(filename)